In [1]:
# Imports
import pandas as pd
import numpy as np
import sklearn
from sklearn import pipeline, ensemble
import os

In [2]:
# File Reading and Splitting

# Read files
data = []
labels = []
longest_sample_length = 0

for root, dirpath, files in os.walk("Spectrograms"):
        for folder in dirpath:
            for r, d, files_in_folder in os.walk(os.path.join("Spectrograms", folder)):
                for f in files_in_folder:
                    path = os.path.join(root, folder, f)
                    d = np.load(path)
                    
                    
                    if len(d[1]) > longest_sample_length:
                        longest_sample_length = len(d[1])
                    
                    data.append(d)
                    labels.append(folder)
                    

data_new = []
for i in range(len(data)):
    d = data[i][1]
    num_samp = longest_sample_length // len(d)
    remainder = longest_sample_length % len(d)
    
    temp = []
    for j in range(num_samp):
        temp.extend(d)
    temp.extend(d[:remainder])
    
    data_new.append(temp)

print("Finished converting and padding data")

dx = [data_new, labels]
dx = np.swapaxes(dx, 0, 1)

split2 = 10 / 90
train_val, test = sklearn.model_selection.train_test_split(dx, test_size=0.10, random_state=42)
train, val = sklearn.model_selection.train_test_split(train_val, test_size=split2)

print("Split Complete! Train Size:", len(train), "Validation Size:", len(val), "Test Size:", len(test))

Finished converting and padding data
Split Complete! Train Size: 4831 Validation Size: 604 Test Size: 604


/opt/conda/envs/common/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [3]:
# Rotate all on axes
train = np.swapaxes(train, 0, 1)
val = np.swapaxes(val, 0, 1)

In [4]:
tdf = pd.DataFrame()
vdf = pd.DataFrame()

for i in range(len(train[0])):
    tdf[i] = train[0][i]
for i in range(len(val[0])):
    vdf[i] = val[0][i]
    
tdf = np.swapaxes(tdf, 0, 1)
vdf = np.swapaxes(vdf, 0, 1)

tdf.head()

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
0,-132.094803,-162.729691,-209.586823,-206.846161,-210.168121,-209.382629,-208.077423,-208.119507,-207.000534,-205.710861,...,-222.967743,-233.924774,-237.872192,-238.970673,-238.261276,-246.124283,-231.542267,-214.821625,-203.343384,-143.860199
1,21.410828,26.528183,29.129637,29.149458,24.201439,23.332161,28.917179,33.199890,30.809908,23.755062,...,-20.878052,-79.153580,-94.438782,-87.411804,-93.428322,-99.735306,-95.444016,-105.520828,-109.880737,-105.496628
2,-60.578468,-61.670719,-56.146461,-50.563156,-52.938301,-57.549492,-57.147354,-58.978863,-61.489365,-64.685242,...,-58.470966,-62.843857,-60.287483,-51.780205,-47.185745,-50.389664,-51.820774,-43.087669,-51.153271,-57.654480
3,35.505333,1.254894,-16.564039,-12.587234,-20.076344,-25.260284,-22.994654,-15.913017,0.146202,-11.410685,...,0.996404,-21.649406,-29.294886,-26.554008,-27.082706,-26.679571,-20.792717,-7.018718,-6.644827,-31.949131
4,31.026970,31.729858,25.249443,27.608204,28.268288,22.603836,28.572128,34.499481,31.683598,34.070274,...,31.386410,37.243145,38.587780,31.751575,33.644600,39.076935,32.283306,31.610254,41.408352,33.352432


In [5]:
pipeline = sklearn.pipeline.make_pipeline(sklearn.ensemble.RandomForestClassifier())

X = tdf
y = train[1]

pipeline.fit(X, y)

valpred = pipeline.predict(vdf)

v_acc = sklearn.metrics.accuracy_score(valpred, val[1])

print("Validation:", v_acc)

Validation: 0.6175496688741722
